In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
model = hub.load("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow1/openimages-v4-ssd-mobilenet-v2/1").signatures["default"]

In [ ]:
import random 
import math

In [ ]:
colorcodes = {}

In [ ]:

def drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color):
    im_height, im_width, _  = image.shape
    left,top,right,bottom = int(xmin*im_width), int(ymin*im_height), int(xmax*im_width),int(ymax*im_height)
    cv2.rectangle(image,(left,top),(right,bottom),color = color,thickness = 4)
    FONT_SCALE = 5e-3
    THICKNESS_SCALE = 4e-3
    width = right-left
    height = bottom-top
    TEXT_Y_OFFSET_SCALE = 1e-2
    cv2.rectangle(
        image,
        (left,top- int(height * 6e-2)),
        (right,top),
        color = color,
        thickness = -1
        
    )
    cv2.putText(
        image,
        namewithscore,
        (left,top-int(height * TEXT_Y_OFFSET_SCALE)),
        fontFace = cv2.FONT_HERSHEY_PLAIN,
        fontScale = min(width,height)* FONT_SCALE,
        thickness = math.ceil(min(width,height)* THICKNESS_SCALE),
        color = (255,255,255)
    )

In [ ]:
def draw(image,boxes,classnames,scores):
    boxesidx = tf.image.non_max_suppression(boxes,scores,max_output_size = 4, iou_threshold = 0.5,score_threshold = 0.1)
#     for i in range(len(boxes)):
    for i in boxesidx:
        ymin,xmin,ymax,xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0,255,30)
            c2 = random.randrange(0,255,25)
            c3 = random.randrange(0,255,50)
            colorcodes.update({classname: (c1,c2,c3)})
            color = colorcodes[classname]
        namewithscore = "{}:{}".format(classname,int(100*scores[i]))
        drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color)
        
    return image

In [ ]:
image = cv2.imread("04.png")
#image = cv2.resize(image,(800,600)
image2 = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
converted_img = tf.image.convert_image_dtype(image2,tf.float32)[tf.newaxis, ...]

In [ ]:
detection = model(converted_img)

In [ ]:
print(detection["detection_class_entities"])

In [ ]:
result = {key:value.numpy() for key, value in detection.items()}
imagewithboxes = draw(image,result['detection_boxes'], result['detection_class_entities'], result["detection_scores"])
imagewithboxes = cv2.resize(imagewithboxes,(0,0),fx=0.7,fy=0.7)
#cv2.imshow("image2", imagewithboxes)
#cv2.waitKey(0)
cv2.imwrite("detected.jpg",imagewithboxes)

In [ ]:

result = {key:value.numpy() for key,value in detection.items()}
imagewithboxes = draw(image,result['detection_boxes'],result['detection_class_entities'],result["detection_scores"])
cv2.imshow("image",imagewithboxes)
cv2.waitKey(0)

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub
import random
import math

# Load the model
model = hub.load("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow1/openimages-v4-ssd-mobilenet-v2/1").signatures["default"]

# Initialize a dictionary to store color codes
colorcodes = {}

# Function to draw bounding boxes
def drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color):
    im_height, im_width, _ = image.shape
    left, top, right, bottom = int(xmin * im_width), int(ymin * im_height), int(xmax * im_width), int(ymax * im_height)
    cv2.rectangle(image, (left, top), (right, bottom), color=color, thickness=2)  # Adjusted thickness

    FONT_SCALE = 0.5  # Adjusted font scale
    THICKNESS_SCALE = 1  # Adjusted thickness scale
    TEXT_Y_OFFSET_SCALE = 0.01

    # Draw the background rectangle for text
    cv2.rectangle(
        image,
        (left, top - int(20)),  # Adjusted the height of the rectangle
        (right, top),
        color=color,
        thickness=-1
    )

    # Put the label text on the image
    cv2.putText(
        image,
        namewithscore,
        (left, top - int(5)),  # Adjusted the position of the text
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=FONT_SCALE,
        thickness=THICKNESS_SCALE,
        color=(255, 255, 255)
    )

# Function to process detections and draw them on the image
def draw(image, boxes, classnames, scores):
    # Filter detections based on scores (lowered the score threshold to 0.3)
    boxesidx = tf.image.non_max_suppression(boxes, scores, max_output_size=10, iou_threshold=0.4, score_threshold=0.3)

    for i in boxesidx:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            # Generate random colors for new classes
            c1 = random.randrange(0, 255, 30)
            c2 = random.randrange(0, 255, 25)
            c3 = random.randrange(0, 255, 50)
            colorcodes.update({classname: (c1, c2, c3)})
            color = colorcodes[classname]

        # Prepare the label text with class and score
        namewithscore = "{}:{}".format(classname, int(100 * scores[i]))
        drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color)

    return image

# Load the image
image = cv2.imread("04.png")

# Check if the image was loaded successfully
if image is None:
    print("Error: Image not found or path is incorrect.")
else:
    # Resize the image for consistent processing (make the dimensions proportional)
    height, width = 800, 600  # Adjust the size for display
    image = cv2.resize(image, (width, height))
    
    # Convert the image from BGR to RGB as the model expects RGB input
    image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Normalize the image and add batch dimension
    converted_img = tf.image.convert_image_dtype(image2, tf.float32)[tf.newaxis, ...]

    # Perform detection
    detection = model(converted_img)

    # Extract detection results
    result = {key: value.numpy() for key, value in detection.items()}

    # Draw bounding boxes and labels on the image
    imagewithboxes = draw(image, result['detection_boxes'], result['detection_class_entities'], result["detection_scores"])

    # Display the image with bounding boxes
    cv2.imshow("image", imagewithboxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()  # Clean up the window after displaying the image


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import tensorflow_hub as hub
import random
import math
from IPython.display import display, Image, clear_output

# Load the model
model = hub.load("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow1/openimages-v4-ssd-mobilenet-v2/1").signatures["default"]

# Initialize a dictionary to store color codes
colorcodes = {}

# Function to draw bounding boxes
def drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color):
    im_height, im_width, _ = image.shape
    left, top, right, bottom = int(xmin * im_width), int(ymin * im_height), int(xmax * im_width), int(ymax * im_height)
    cv2.rectangle(image, (left, top), (right, bottom), color=color, thickness=2)

    FONT_SCALE = 0.5
    THICKNESS_SCALE = 1
    TEXT_Y_OFFSET_SCALE = 0.01

    # Draw the background rectangle for text
    cv2.rectangle(
        image,
        (left, top - int(20)),
        (right, top),
        color=color,
        thickness=-1
    )

    # Put the label text on the image
    cv2.putText(
        image,
        namewithscore,
        (left, top - int(5)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=FONT_SCALE,
        thickness=THICKNESS_SCALE,
        color=(255, 255, 255)
    )

# Function to process detections and draw them on the image
def draw(image, boxes, classnames, scores):
    # Filter detections based on scores (lowered the score threshold to 0.3)
    boxesidx = tf.image.non_max_suppression(boxes, scores, max_output_size=10, iou_threshold=0.4, score_threshold=0.3)

    for i in boxesidx:
        ymin, xmin, ymax, xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0, 255, 30)
            c2 = random.randrange(0, 255, 25)
            c3 = random.randrange(0, 255, 50)
            colorcodes.update({classname: (c1, c2, c3)})
            color = colorcodes[classname]

        # Prepare the label text with class and score
        namewithscore = "{}:{}".format(classname, int(100 * scores[i]))
        drawbox(image, ymin, xmin, ymax, xmax, namewithscore, color)

    return image

# Real-time object detection
def detect_objects_in_real_time():
    cap = cv2.VideoCapture(0)  # Start video capture from the webcam

    while True:
        ret, frame = cap.read()  # Capture frame-by-frame
        if not ret:
            break

        # Resize and convert the frame to RGB
        height, width = 400, 400
        frame = cv2.resize(frame, (width, height))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        converted_img = tf.image.convert_image_dtype(image_rgb, tf.float32)[tf.newaxis, ...]

        # Perform detection
        detection = model(converted_img)

        # Extract detection results
        result = {key: value.numpy() for key, value in detection.items()}

        # Draw bounding boxes on the frame
        image_with_boxes = draw(frame, result['detection_boxes'], result['detection_class_entities'], result["detection_scores"])

        # Display the resulting frame in the notebook
        clear_output(wait=True)
        _, encoded_image = cv2.imencode('.jpg', image_with_boxes)
        display(Image(data=encoded_image))

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()  # Release the webcam

# Run the real-time object detection
detect_objects_in_real_time()
